# Train the Model to Run

In [ ]:
def nn_step_fn(carry, _):
    data, key, nn_params, model = carry
    new_data, new_key = nn_mjx_one_step(nn_params, model, data, key)
    new_carry = (new_data, new_key, nn_params, model)
    head_hight = new_data.sensordata[2]
    # jax.debug.print("Head Height {0}",head_hight)
    return new_carry, (head_hight-1.63)**2
    
def decay_sum_scan(x, decay):
    def f(sxtm1, xt):
        b = xt + decay * sxtm1
        return b, b
    return jax.lax.scan(f, jp.zeros(x.shape[1:]), x)[1]

def reward_n_step(nn_params, model, data, key):
    repeat_n = 150
    _, head_heights = jax.lax.scan(nn_step_fn, (data, key, nn_params, model), None, repeat_n)
    reward = decay_sum_scan(head_heights, 0.95)[repeat_n -1]
    return reward
    
# reward_grad = jax.jit(jax.grad(reward_n_step))


def batch_reward(nn_params, batched_data, keys, model):
    return jp.mean(jax.vmap(reward_n_step, (None, None, 0, 0))(nn_params, model, batched_data, keys))


jit_batch_reward_grad = jax.jit(jax.grad((batch_reward)))
jit_batch_reward = jax.jit(batch_reward)

# The partial function will make model a static member. However, it will not reduce the compile time.
# partial_batch_reward = partial(batch_reward, model = mjx_model)
# jit_batch_reward_grad = jax.jit(jax.grad((partial_batch_reward)))
# jit_batch_reward = jax.jit(partial_batch_reward)